## INTRO

- USE_CASE and ORGANIZATION had values with small counts, which were bundled into 'other' category
- 'Classification Count' to be replaced was adjusted
- After looking at the original data, Name was determined to be a an important part of data as there are numerous institutions that repeatedly applied, so after looking at count values, 'NAME' was  included as a part of input, where value counts less than 5 were bundled into 'other' category
- Keras model was adjusted to reflect the new data
- Epochs were increased to 100

## Preprocessing

In [17]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import sklearn as skl

In [113]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [114]:
# Drop the non-beneficial ID columns, 'EIN', 'NAME', 'STATUS' and 'SPECIAL_CONSIDERATIONS'
application_df = application_df.drop(["EIN","SPECIAL_CONSIDERATIONS","STATUS"],1)
application_df.head()

C:\Users\ezgib\AppData\Local\Temp\ipykernel_21160\1439037256.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(["EIN","SPECIAL_CONSIDERATIONS","STATUS"],1)


,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [115]:
#Checking to see the unique values count for special_considerations
application_df["USE_CASE"].value_counts()

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [116]:
#Checking to see the unique values count for special_considerations
application_df["ORGANIZATION"].value_counts()

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [117]:
#Checking to see the unique values count for special_considerations
application_df["AFFILIATION"].value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [118]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T9","T13","T12","T2","T25","T14","T29","T15","T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [119]:
# Choose a cutoff value and create a list of affliation types to be replaced
# use the variable name `affliation_types_to_replace`
affliation_types_to_replace = ["Family/Parent", "National", "Regional", "Other"]

# Replace in dataframe
for aff in affliation_types_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(aff,"Other")

# Check to make sure binning was successful
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [120]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df["CLASSIFICATION"].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [121]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts.loc[classification_counts < 200].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

In [122]:
# Choose a cutoff value and create a list of USE_CASE types to be replaced
# use the variable name `usecase_types_to_replace`
usecase_types_to_replace = ["CommunityServ", "Heathcare", "Other"]

# Replace in dataframe
for usec in usecase_types_to_replace:
    application_df['USE_CASE'] = application_df['USE_CASE'].replace(usec,"Other")

# Check to make sure binning was successful
application_df['USE_CASE'].value_counts()

Preservation    28095
ProductDev       5671
Other             533
Name: USE_CASE, dtype: int64

In [123]:
# Choose a cutoff value and create a list of ORGANIZATION types to be replaced
# use the variable name `usecase_types_to_replace`
organization_types_to_replace = ["Co-operative", "Corporation"]

# Replace in dataframe
for org in organization_types_to_replace:
    application_df['ORGANIZATION'] = application_df['ORGANIZATION'].replace(org,"Co-operative")

# Check to make sure binning was successful
application_df['ORGANIZATION'].value_counts()

Trust           23515
Association     10255
Co-operative      529
Name: ORGANIZATION, dtype: int64

In [124]:
# Look at NAME value counts for binning
name_count = application_df['NAME'].value_counts()
name_count

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64

In [130]:
len(name_count)

19568

In [134]:
len(name_count[name_count>5])

354

In [135]:
# Determine which values to replace if counts are less than 5
names_to_replace = list(name_count[name_count < 5].index)

# Replace in dataframe
for names in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(names,"Other")
    
# Check to make sure binning was successful
application_df['NAME'].value_counts()

Other                                                                19803
PARENT BOOSTER USA INC                                                1260
TOPS CLUB INC                                                          765
UNITED STATES BOWLING CONGRESS INC                                     700
WASHINGTON STATE UNIVERSITY                                            492
                                                                     ...  
GUILDS OF THE SANTA FE OPERA INC                                         5
FAMILY MOTOR COACH ASSOCIATION                                           5
JAMESTOWN SOCIETY                                                        5
MOST WORSHIPFUL PRINCE HALL GRAND LODGE OF FREE & ACCEPTED MASONS        5
PTA HAWAII CONGRESS                                                      5
Name: NAME, Length: 403, dtype: int64

In [136]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTS MINISTRY,NAME_ACTS MISSIONS,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [137]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop('IS_SUCCESSFUL', axis=1).values
y = application_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [138]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [139]:
len(X_train[0])

439

## Compile, Train and Evaluate the Model

In [140]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=40,
        step=5), activation=activation, input_dim=len(X_train[0])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=40,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [141]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [142]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 57s]
val_accuracy: 0.7958017587661743

Best val_accuracy So Far: 0.7967346906661987
Total elapsed time: 00h 18m 41s
INFO:tensorflow:Oracle triggered exit


In [146]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 26,
 'num_layers': 3,
 'units_0': 21,
 'units_1': 6,
 'units_2': 36,
 'units_3': 26,
 'units_4': 1,
 'units_5': 11,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0012'}

In [163]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  25
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 25)                11000     
                                                                 
 dense_24 (Dense)            (None, 30)                780       
                                                                 
 dense_25 (Dense)            (None, 10)                310       
                                                                 
 dense_26 (Dense)            (None, 1)                 11        
                                                                 
Total params: 12,101
Trainable params: 12,101
Non-trainable params: 0
_________________________________________________________________


In [164]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [165]:
# Train the model
fit_model = nn_model.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4745 - accuracy: 0.7716
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4313 - accuracy: 0.7932
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4261 - accuracy: 0.7955
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4230 - accuracy: 0.7975
Epoch 5/100
804/804 [==============================] - 3s 3ms/step - loss: 0.4213 - accuracy: 0.7974
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4195 - accuracy: 0.7978
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.4185 - accuracy: 0.7998
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4173 - accuracy: 0.8013
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4166 - accuracy: 0.8002
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4156 - accura

804/804 [==============================] - 2s 2ms/step - loss: 0.3978 - accuracy: 0.8093
Epoch 82/100
804/804 [==============================] - 3s 3ms/step - loss: 0.3974 - accuracy: 0.8092
Epoch 83/100
804/804 [==============================] - 1s 2ms/step - loss: 0.3978 - accuracy: 0.8105
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 0.3974 - accuracy: 0.8095
Epoch 85/100
804/804 [==============================] - 2s 2ms/step - loss: 0.3975 - accuracy: 0.8099
Epoch 86/100
804/804 [==============================] - 2s 2ms/step - loss: 0.3974 - accuracy: 0.8098
Epoch 87/100
804/804 [==============================] - 3s 4ms/step - loss: 0.3977 - accuracy: 0.8082
Epoch 88/100
804/804 [==============================] - 3s 4ms/step - loss: 0.3974 - accuracy: 0.8092
Epoch 89/100
804/804 [==============================] - 3s 3ms/step - loss: 0.3972 - accuracy: 0.8101
Epoch 90/100
804/804 [==============================] - 2s 2ms/step - loss: 0.3973 - accuracy: 

In [166]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4526 - accuracy: 0.7929 - 674ms/epoch - 3ms/step
Loss: 0.45258042216300964, Accuracy: 0.7928863167762756


In [167]:
# Export our model to HDF5 file
nn_model.save_weights('AlphabetSoupCharity_Optimization_Final.h5')